In [1]:
import pandas as pd
import numpy as np
import zarr
from polaris.experimental._dataset_v2 import DatasetV2
import numcodecs

/opt/conda/envs/plumber_polaris_2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv('train.csv')
train

,smiles,ki,ic50,kd,ec50,sequence,ki_sign,ic50_sign,kd_sign,ec50_sign,uniprot_id,source,split
0,BP(=O)(OCC1C=CC(n2cc(C)c(=O)[nH]c2=O)O1)OP(=O)...,45.0,NaN,NaN,NaN,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,=,NaN,NaN,NaN,Q72547,bdb,train
1,BP(=O)(OCC1C=CC(n2cnc3c(=O)[nH]c(N)nc32)C1)OP(...,34.0,NaN,NaN,NaN,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,=,NaN,NaN,NaN,Q72547,bdb,train
2,BP(=O)(OCC1CCC(n2cc(C)c(=O)[nH]c2=O)O1)OP(=O)(...,236.0,NaN,NaN,NaN,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,=,NaN,NaN,NaN,Q72547,bdb,train
3,BP(=O)(OCC1CCC(n2ccc(=O)[nH]c2=O)O1)OP(=O)(O)C...,438.0,NaN,NaN,NaN,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,=,NaN,NaN,NaN,Q72547,bdb,train
4,BP(=O)(OCC1CCC(n2ccc(N)nc2=O)O1)OP(=O)(O)C(F)(...,23.0,NaN,NaN,NaN,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,=,NaN,NaN,NaN,Q72547,bdb,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861735,c1nnc(NCc2ccc3c(c2)OCO3)[nH]1,NaN,17940.0,NaN,NaN,MYFSSLCKFLPISEKEKIYLNIVKKRFCKSNIYYNNNNNNIINYNK...,NaN,=,NaN,NaN,Q8IL11,bdb,train
1861736,c1nnc(SSc2nncs2)s1,NaN,560.0,NaN,NaN,MADQAPFDTDVNTLTRFVMEEGRKARGTGELTQLLNSLCTAVKAIS...,NaN,=,NaN,NaN,P09467,bdb,train
1861737,c1nnc2n1CCCC2,NaN,244000.0,NaN,NaN,MARTTSQLYDAVPIQSSVVLCSCPSPSMVRTQTESSTPPGIPGGSR...,NaN,=,NaN,NaN,O15530,bdb,train
1861738,c1nnc2nnc(N3CCOCC3)nn12,NaN,11770.0,NaN,NaN,MKAPAVLAPGILVLLFTLVQRSNGECKEALAKSEMNVNMKYQLPNF...,NaN,=,NaN,NaN,P08581,bdb,train


In [3]:
train['is_active'] = np.nan

In [4]:
test = pd.read_csv('test.csv')
test

,smiles,sequence,uniprot_id,source,split,is_active
0,B[P@@]1(=O)OCC2OC(n3cnc4c(=O)[nH]c(N)nc43)C(O[...,MPHSSLHPSIPCPRGHGAQKAALVLLSACLVTLWGLGEPPEHTLRY...,Q86WV6,bdb,test,1.0
1,B[P@@]1(=O)OCC2OC(n3cnc4c(=O)[nH]c(N)nc43)C(O[...,MPHSSLHPSIPCPRGHGAQKAALVLLSACLVTLWGLGEPPEHTLRY...,Q86WV6,bdb,test,1.0
2,B[P@]1(=O)OCC2OC(n3cnc4c(=O)[nH]c(N)nc43)C(O[P...,MPHSSLHPSIPCPRGHGAQKAALVLLSACLVTLWGLGEPPEHTLRY...,Q86WV6,bdb,test,1.0
3,Brc1cccc(COc2ccc(C3C[C@@H]3NCCC3CCCCC3)cc2)c1,MSNKCDVVVVGGGISGMAAAKLLHDSGLNVVVLEARDRVGGRTYTL...,P27338,bdb,test,1.0
4,Brc1cccc(COc2ccc3c(c2)OCCO3)c1,MSNKCDVVVVGGGISGMAAAKLLHDSGLNVVVLEARDRVGGRTYTL...,P27338,bdb,test,1.0
...,...,...,...,...,...,...
14366,c1nc(NCC2CCOC2)c2cc(-c3ccc4c(c3)OCO4)ccc2n1,MASSSVPPATVSAATAGPGPGFGFASKTKKKHFVQQKVKVFRAADP...,Q8TBX8,bdb,test,1.0
14367,c1nc2cc(NCc3ccc4c(c3)OCCO4)cnc2[nH]1,MAGGRHRRVVGTLHLLLLVAALPWASRGVSPSASAWPEEKNYHQPA...,Q16769,bdb,test,1.0
14368,c1nc2cc(NCc3ccsc3)cnc2[nH]1,MAGGRHRRVVGTLHLLLLVAALPWASRGVSPSASAWPEEKNYHQPA...,Q16769,bdb,test,0.0
14369,c1nc2ccc(-c3nnc(CCc4ccc5c(c4)OCO5)o3)cc2[nH]1,MAGGRHRRVVGTLHLLLLVAALPWASRGVSPSASAWPEEKNYHQPA...,Q16769,bdb,test,1.0


In [5]:
missed_columns = [
    'ki', 'kd', 'ic50', 'ec50', 'ki_sign', 'kd_sign', 'ic50_sign', 'ec50_sign',
]
for column in missed_columns:
    test[column] = np.nan
test

,smiles,sequence,uniprot_id,source,split,is_active,ki,kd,ic50,ec50,ki_sign,kd_sign,ic50_sign,ec50_sign
0,B[P@@]1(=O)OCC2OC(n3cnc4c(=O)[nH]c(N)nc43)C(O[...,MPHSSLHPSIPCPRGHGAQKAALVLLSACLVTLWGLGEPPEHTLRY...,Q86WV6,bdb,test,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B[P@@]1(=O)OCC2OC(n3cnc4c(=O)[nH]c(N)nc43)C(O[...,MPHSSLHPSIPCPRGHGAQKAALVLLSACLVTLWGLGEPPEHTLRY...,Q86WV6,bdb,test,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B[P@]1(=O)OCC2OC(n3cnc4c(=O)[nH]c(N)nc43)C(O[P...,MPHSSLHPSIPCPRGHGAQKAALVLLSACLVTLWGLGEPPEHTLRY...,Q86WV6,bdb,test,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Brc1cccc(COc2ccc(C3C[C@@H]3NCCC3CCCCC3)cc2)c1,MSNKCDVVVVGGGISGMAAAKLLHDSGLNVVVLEARDRVGGRTYTL...,P27338,bdb,test,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Brc1cccc(COc2ccc3c(c2)OCCO3)c1,MSNKCDVVVVGGGISGMAAAKLLHDSGLNVVVLEARDRVGGRTYTL...,P27338,bdb,test,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14366,c1nc(NCC2CCOC2)c2cc(-c3ccc4c(c3)OCO4)ccc2n1,MASSSVPPATVSAATAGPGPGFGFASKTKKKHFVQQKVKVFRAADP...,Q8TBX8,bdb,test,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14367,c1nc2cc(NCc3ccc4c(c3)OCCO4)cnc2[nH]1,MAGGRHRRVVGTLHLLLLVAALPWASRGVSPSASAWPEEKNYHQPA...,Q16769,bdb,test,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14368,c1nc2cc(NCc3ccsc3)cnc2[nH]1,MAGGRHRRVVGTLHLLLLVAALPWASRGVSPSASAWPEEKNYHQPA...,Q16769,bdb,test,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14369,c1nc2ccc(-c3nnc(CCc4ccc5c(c4)OCO5)o3)cc2[nH]1,MAGGRHRRVVGTLHLLLLVAALPWASRGVSPSASAWPEEKNYHQPA...,Q16769,bdb,test,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
test = test[train.columns]
data = pd.concat([train, test], ignore_index=True)
data

,smiles,ki,ic50,kd,ec50,sequence,ki_sign,ic50_sign,kd_sign,ec50_sign,uniprot_id,source,split,is_active
0,BP(=O)(OCC1C=CC(n2cc(C)c(=O)[nH]c2=O)O1)OP(=O)...,45.0,NaN,NaN,NaN,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,=,NaN,NaN,NaN,Q72547,bdb,train,NaN
1,BP(=O)(OCC1C=CC(n2cnc3c(=O)[nH]c(N)nc32)C1)OP(...,34.0,NaN,NaN,NaN,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,=,NaN,NaN,NaN,Q72547,bdb,train,NaN
2,BP(=O)(OCC1CCC(n2cc(C)c(=O)[nH]c2=O)O1)OP(=O)(...,236.0,NaN,NaN,NaN,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,=,NaN,NaN,NaN,Q72547,bdb,train,NaN
3,BP(=O)(OCC1CCC(n2ccc(=O)[nH]c2=O)O1)OP(=O)(O)C...,438.0,NaN,NaN,NaN,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,=,NaN,NaN,NaN,Q72547,bdb,train,NaN
4,BP(=O)(OCC1CCC(n2ccc(N)nc2=O)O1)OP(=O)(O)C(F)(...,23.0,NaN,NaN,NaN,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...,=,NaN,NaN,NaN,Q72547,bdb,train,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876106,c1nc(NCC2CCOC2)c2cc(-c3ccc4c(c3)OCO4)ccc2n1,NaN,NaN,NaN,NaN,MASSSVPPATVSAATAGPGPGFGFASKTKKKHFVQQKVKVFRAADP...,NaN,NaN,NaN,NaN,Q8TBX8,bdb,test,1.0
1876107,c1nc2cc(NCc3ccc4c(c3)OCCO4)cnc2[nH]1,NaN,NaN,NaN,NaN,MAGGRHRRVVGTLHLLLLVAALPWASRGVSPSASAWPEEKNYHQPA...,NaN,NaN,NaN,NaN,Q16769,bdb,test,1.0
1876108,c1nc2cc(NCc3ccsc3)cnc2[nH]1,NaN,NaN,NaN,NaN,MAGGRHRRVVGTLHLLLLVAALPWASRGVSPSASAWPEEKNYHQPA...,NaN,NaN,NaN,NaN,Q16769,bdb,test,0.0
1876109,c1nc2ccc(-c3nnc(CCc4ccc5c(c4)OCO5)o3)cc2[nH]1,NaN,NaN,NaN,NaN,MAGGRHRRVVGTLHLLLLVAALPWASRGVSPSASAWPEEKNYHQPA...,NaN,NaN,NaN,NaN,Q16769,bdb,test,1.0


In [7]:
data['ki_sign'] = data['ki_sign'].fillna("")
data['ic50_sign'] = data['ic50_sign'].fillna("")
data['kd_sign'] = data['kd_sign'].fillna("")
data['ec50_sign'] = data['ec50_sign'].fillna("")

In [10]:
plumber_zarr = 'plumber.zarr'
n = len(data)

root = zarr.open(plumber_zarr, mode='w')
root.empty('smiles', shape=n, dtype=object, object_codec=numcodecs.VLenUTF8())
root.empty('ki', shape=n, dtype=float)
root.empty('ic50', shape=n, dtype=float)
root.empty('kd', shape=n, dtype=float)
root.empty('ec50', shape=n, dtype=float)
root.empty('sequence', shape=n, dtype=object, object_codec=numcodecs.VLenUTF8())
root.empty('ki_sign', shape=n, dtype=object, object_codec=numcodecs.VLenUTF8())
root.empty('ic50_sign', shape=n, dtype=object, object_codec=numcodecs.VLenUTF8())
root.empty('kd_sign', shape=n, dtype=object, object_codec=numcodecs.VLenUTF8())
root.empty('ec50_sign', shape=n, dtype=object, object_codec=numcodecs.VLenUTF8())
root.empty('uniprot_id', shape=n, dtype=object, object_codec=numcodecs.VLenUTF8())
root.empty('source', shape=n, dtype=object, object_codec=numcodecs.VLenUTF8())
root.empty('split', shape=n, dtype=object, object_codec=numcodecs.VLenUTF8())
root.empty('is_active', shape=n, dtype=float)

for col in train.columns:
    root[col][:] = data[col].values

In [12]:
zarr.convenience.consolidate_metadata(plumber_zarr)

<zarr.hierarchy.Group '/'>

In [13]:
readme = """
PLUMBER is a benchmark for developing sequence-based models for binding affinity prediction, based on the PLINDER benchmark. The test data points include the SMILES of a small ligand, the sequence of the monomeric target protein, and a binary activity label corresponding to Ki/Kd < 1 μM. PLINDER is employed to split the proteins into training and testing sets. To enhance flexibility, the training set includes continuous values and their corresponding signs (=, >, <) and is compiled from various sources: ChEMBL, BindingDB, and BioLip2.

PLUMBER contains the following columns:

- `SMILES`: the SMILES representation of a molecule
- `sequence`: the amino acid sequence of a monomer target protein
- `uniprot_id`: the UniProt ID of that protein
- `source`: either chembl, bdb, or biolip
- `split`: either 'train' or 'test
- `is_active`: a binary label indicating if the molecule has a Ki/Kd < 1 μM

The train rows also have `ki`, `kd`, `ic50`, and `ec50` activity values, while `ki_sign`, `kd_sign`, `ic50_sign`, and `ec50_sign` specify the corresponding relationship, such as =, <, or >. You can choose to use in training only Ki/Kd equality data, but alternative strategies can incorporate the other activity types as well.
"""

plumber_license = "CC-BY-4.0" # Should be one of those: "CC-BY-4.0", "CC-BY-SA-4.0", "CC-BY-NC-4.0", "CC-BY-NC-SA-4.0", "CC0-1.0", "MIT"

dataset = DatasetV2(
    zarr_root_path=plumber_zarr,
    name="plumber", 
    description="Binding affinity prediction benchmark for sequence-based models", 
    owner="bioptic",
    tags=["sequence", "binding affinity", "sbdd"],
    readme=readme,
    license=plumber_license,
)